In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
pd.set_option('display.max_columns', 500)

In [33]:
north = -6.9875
west = -37.3303
south = -7.0882
east = -37.2215
pois = ox.geometries.geometries_from_bbox(north, south, east, west, tags={'amenity':True})
pois = pois[['amenity', 'name', 'geometry', 'opening_hours']]
pois

amenity                        name  \
element_type osmid                                                      
node         1798496605              fuel                 Posto Viola   
             1798496642              taxi  Moto táxi Praça dos Pombos   
             1798525769              taxi       Táxi Rodoviária Velha   
             1798525772              taxi    Táxi Fórum Miguel Sátiro   
             1798919196               bar                 Bar do Rock   
...                                   ...                         ...   
way          1084925006  place_of_worship                         NaN   
             1085721565  place_of_worship    Capela da Cruz da Menina   
             1098137716            school                         NaN   
             1112595201  place_of_worship                         NaN   
relation     7375603          marketplace             Mercado Público   

                                                                  geometry  \
element_type osmid                                                           
node         1798496605                         POINT (-37.27847 -7.02390)   
             1798496642                         POINT (-37.27843 -7.02361)   
             1798525769                         POINT (-37.27735 -7.02265)   
             1798525772                         POINT (-37.27702 -7.02601)   
             1798919196                         POINT (-37.27702 -7.02204)   
...                                                                    ...   
way          1084925006  POLYGON ((-37.27436 -7.00392, -37.27443 -7.003...   
             1085721565  POLYGON ((-37.30224 -7.00568, -37.30223 -7.005...   
             1098137716  POLYGON ((-37.27944 -7.06101, -37.27924 -7.060...   
             1112595201  POLYGON ((-37.23823 -7.06129, -37.23823 -7.061...   
relation     7375603     POLYGON ((-37.27953 -7.02765, -37.27953 -7.027...   

                                                      opening_hours  
element_type osmid                                                   
node         1798496605                                         NaN  
             1798496642                                         NaN  
             1798525769                                         NaN  
             1798525772                                         NaN  
             1798919196  We-Su 19:00-02:00+; week 25-26 18:00-04:00  
...                                                             ...  
way          1084925006                                         NaN  
             1085721565                                         NaN  
             1098137716                                         NaN  
             1112595201                                         NaN  
relation     7375603                              Mo-Sa 07:00-17:00  

[829 rows x 4 columns]

In [34]:
def get_centroid(geom):
    if type(geom) == 'shapely.geometry.point.Point':
        return geom
    return geom.centroid

pois['geometry'] = pois['geometry'].apply(get_centroid)

In [36]:
pois

amenity                        name  \
element_type osmid                                                      
node         1798496605              fuel                 Posto Viola   
             1798496642              taxi  Moto táxi Praça dos Pombos   
             1798525769              taxi       Táxi Rodoviária Velha   
             1798525772              taxi    Táxi Fórum Miguel Sátiro   
             1798919196               bar                 Bar do Rock   
...                                   ...                         ...   
way          1084925006  place_of_worship                         NaN   
             1085721565  place_of_worship    Capela da Cruz da Menina   
             1098137716            school                         NaN   
             1112595201  place_of_worship                         NaN   
relation     7375603          marketplace             Mercado Público   

                                           geometry  \
element_type osmid                                    
node         1798496605  POINT (-37.27847 -7.02390)   
             1798496642  POINT (-37.27843 -7.02361)   
             1798525769  POINT (-37.27735 -7.02265)   
             1798525772  POINT (-37.27702 -7.02601)   
             1798919196  POINT (-37.27702 -7.02204)   
...                                             ...   
way          1084925006  POINT (-37.27438 -7.00381)   
             1085721565  POINT (-37.30223 -7.00561)   
             1098137716  POINT (-37.27949 -7.06073)   
             1112595201  POINT (-37.23818 -7.06126)   
relation     7375603     POINT (-37.27903 -7.02779)   

                                                      opening_hours  
element_type osmid                                                   
node         1798496605                                         NaN  
             1798496642                                         NaN  
             1798525769                                         NaN  
             1798525772                                         NaN  
             1798919196  We-Su 19:00-02:00+; week 25-26 18:00-04:00  
...                                                             ...  
way          1084925006                                         NaN  
             1085721565                                         NaN  
             1098137716                                         NaN  
             1112595201                                         NaN  
relation     7375603                              Mo-Sa 07:00-17:00  

[829 rows x 4 columns]

In [37]:
# pois.to_json()
pois.to_file('pois_from_patos.geojson', driver="GeoJSON") 

In [38]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts="http://localhost:9200", http_auth=('elastic', 'changeme'), verify_certs=False)

/tmp/ipykernel_4153/3962933453.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts="http://localhost:9200", http_auth=('elastic', 'changeme'), verify_certs=False)


In [39]:
es.info()

ObjectApiResponse({'name': 'elk', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'q7oWD5V7RWyCXRr4UqUt9g', 'version': {'number': '8.5.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '4ed5ee9afac63de92ec98f404ccbed7d3ba9584e', 'build_date': '2022-12-05T18:22:22.226119656Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [40]:
import geojson
with open('pois_from_patos.geojson') as f:
    pois_list = geojson.load(f)
pois_list = pois_list['features']
dict(pois_list[0])

{'type': 'Feature',
 'geometry': {"coordinates": [-37.278472, -7.023898], "type": "Point"},
 'properties': {'element_type': 'node',
  'osmid': 1798496605,
  'amenity': 'fuel',
  'name': 'Posto Viola',
  'opening_hours': None}}

In [42]:
es.indices.delete(index='patos-pois-index')

ObjectApiResponse({'acknowledged': True})

In [43]:
if not es.indices.exists(index='patos-pois-index').body:
   es.indices.create(index='patos-pois-index', ignore=400, mappings= {
        'properties':{
            'geometry': {
                'type': 'geo_point'
            }
        }
    })

/tmp/ipykernel_4153/3383120693.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='patos-pois-index', ignore=400, mappings= {


In [44]:
for i,poi in enumerate(pois_list):
    resp = es.index(index="patos-pois-index", id=i+1, document=poi)
    if i%1000==0:
        print(str(i+1)+'º instance: ',resp['result'])
print('Completed.')

1º instance:  created
Completed.


In [45]:
pois.shape

(829, 4)

In [46]:
es.search(body={"query": {"match_all": {}}}, index = 'patos-pois-index')

/tmp/ipykernel_4153/2386377839.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.search(body={"query": {"match_all": {}}}, index = 'patos-pois-index')


ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 829, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'patos-pois-index', '_id': '1', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.278472, -7.023898]}, 'properties': {'element_type': 'node', 'osmid': 1798496605, 'amenity': 'fuel', 'name': 'Posto Viola', 'opening_hours': None}}}, {'_index': 'patos-pois-index', '_id': '2', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.27843, -7.023605]}, 'properties': {'element_type': 'node', 'osmid': 1798496642, 'amenity': 'taxi', 'name': 'Moto táxi Praça dos Pombos', 'opening_hours': None}}}, {'_index': 'patos-pois-index', '_id': '3', '_score': 1.0, '_source': {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-37.277349, -7.022649]}, 'properties': {'element_type': 'node', 'osmid': 1798

In [47]:
es.indices.get_mapping(index = 'patos-pois-index')

ObjectApiResponse({'patos-pois-index': {'mappings': {'properties': {'geometry': {'type': 'geo_point'}, 'properties': {'properties': {'amenity': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'element_type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'name': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'opening_hours': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'osmid': {'type': 'long'}}}, 'type': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}})

In [ ]:
# query = {
#     "bool": {
#         "filter": {
#             "match_phrase": {
#                 "country": "United States"
#             }
#         },
#         "must_not": {
#             "range": {
#                 "release_year": {"lte": 2015}
#             }
#         }
#     }
# }

# result = es.search(index="netflix_shows", query=query)
# print(result)

# es.delete(index="netflix_shows", id="1337")
# es.indices.delete(index="netflix_shows")